<b>
    Nama: Siti Mawaddah<br>
    NPM: 1908107010031
</b>

<p>Code berikut mengimplementasikan deteksi wajah menggunakan haar cascade. Disini saya memanfaatkan input key dari keyboard.
    <ul>
        <li>Input 'b' akan blur latar belakang</li>
        <li>Input 'f' akan menambah frame di wajah</li>
        <li>Input 'c' akan memberi filter kucing</li>
    </ul>
</p>

In [6]:
import cv2
import numpy as np

face = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

#read filter
cat = cv2.imread('filter/cat.png')

#get shape of cat
original_cat_h,original_cat_w,cat_channels = cat.shape

#convert to gray
cat_gray = cv2.cvtColor(cat, cv2.COLOR_BGR2GRAY)

#create mask and inverse mask of cat
ret, original_mask = cv2.threshold(cat_gray, 10, 255, cv2.THRESH_BINARY_INV)
original_mask_inv = cv2.bitwise_not(original_mask)

# Get the webcam
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
frame_h, frame_w = frame.shape[:2]

blur = False
framed = False
cat_filter = False


while True:
    ret, frame = cap.read()
    if(ret):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        f = face.detectMultiScale(gray,scaleFactor=1.1, minNeighbors=7)
        
        for x,y,w,h in f:
            if blur:
                sub_face = frame[y:y+h, x:x+w]

                frame = cv2.GaussianBlur(frame, (23,23), 30)
                
                frame[y:y+sub_face.shape[0], x:x+sub_face.shape[1]] = sub_face
                
            if framed:
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 2)
                
            if cat_filter:
                #cat size in relation to face by scaling
                cat_width = int(1.5 * w)
                cat_height = int(cat_width * original_cat_h / original_cat_w)

                #setting location of coordinates of cat
                cat_x1 = x+w - int(w/2) - int(cat_width/2)
                cat_x2 = cat_x1 + cat_width
                cat_y1 = y - int(h*0.5)
                cat_y2 = cat_y1 + cat_height 

                #check to see if out of frame
                if cat_x1 < 0:
                    cat_x1 = 0
                if cat_y1 < 0:
                    cat_y1 = 0
                if cat_x2 > frame_w:
                    cat_x2 = frame_w
                if cat_y2 > frame_h:
                    cat_y2 = frame_h

                #Account for any out of frame changes
                cat_width = cat_x2 - cat_x1
                cat_height = cat_y2 - cat_y1

                #resize cat to fit on face
                cat = cv2.resize(cat, (cat_width,cat_height), interpolation = cv2.INTER_AREA)
                mask = cv2.resize(original_mask, (cat_width,cat_height), interpolation = cv2.INTER_AREA)
                mask_inv = cv2.resize(original_mask_inv, (cat_width,cat_height), interpolation = cv2.INTER_AREA)

                #take ROI for cat from background that is equal to size of cat image
                roi = frame[cat_y1:cat_y2, cat_x1:cat_x2]

                #original image in background (bg) where cat is not
                roi_bg = cv2.bitwise_and(roi,roi,mask = mask)
                roi_fg = cv2.bitwise_and(cat,cat,mask=mask_inv)
                dst = cv2.add(roi_bg,roi_fg)

                #put back in original image
                frame[cat_y1:cat_y2, cat_x1:cat_x2] = dst
        
        cv2.imshow('Webcam', frame)
    
    ch = 0xFF &cv2.waitKey(1)
    
    # input 'b' di keyboard untuk blur background
    if ch == ord("b"):
        blur = not blur
    
    # input 'f' di keyboard untuk membuat frame di wajah
    if ch == ord("f"):
        framed = not framed
        
    # input 'c' di keyboard untuk menambah filter kucing
    if ch == ord("c"):
        cat_filter = not cat_filter
        
    # input 'q' di keyboard untuk keluar
    if ch == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()